In [1]:
import numpy as np
from syft.core.tensor.autodp.phi_tensor import PhiTensor
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from syft.core.tensor.fixed_precision_tensor import FixedPrecisionTensor as FPT
from syft.core.adp.data_subject_list import DataSubjectList
from syft.core.tensor.lazy_repeat_array import lazyrepeatarray as lra

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Torch Dataset equivalent

We have to implement:
- padding -> DONE!
- random horizontal flip -> DONE!
- random vertical flip -> DONE!
- random rotation -> Done but not confident about resultant LRA, or what to do with 0 values. -> Might try training the model without random rotations to see how testing accuracy suffers.
- normalize -> DONE!

In [2]:
a = PhiTensor(child=np.random.random((5,5)),data_subjects=np.ones((5,5)), min_vals=0, max_vals=1)

In [3]:
a

PhiTensor(child=FixedPrecisionTensor(child=[[32320 13466 31539 53025 47558]
 [48092 46383 10064 64947  6397]
 [23758 54672 39889 31523 35489]
 [64122 55424 64644 44428  5558]
 [58499 23557 14045 58880 64741]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [4]:
a.pad(width=1)

PhiTensor(child=FixedPrecisionTensor(child=[[46383 48092 46383 10064 64947  6397 64947]
 [13466 32320 13466 31539 53025 47558 53025]
 [46383 48092 46383 10064 64947  6397 64947]
 [54672 23758 54672 39889 31523 35489 31523]
 [55424 64122 55424 64644 44428  5558 44428]
 [23557 58499 23557 14045 58880 64741 58880]
 [55424 64122 55424 64644 44428  5558 44428]]), min_vals=<lazyrepeatarray data: 0 -> shape: (6, 6)>, max_vals=<lazyrepeatarray data: 1 -> shape: (6, 6)>)

In [5]:
a.random_horizontal_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[32320 13466 31539 53025 47558]
 [48092 46383 10064 64947  6397]
 [23758 54672 39889 31523 35489]
 [64122 55424 64644 44428  5558]
 [58499 23557 14045 58880 64741]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [6]:
a.random_vertical_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[58499 23557 14045 58880 64741]
 [64122 55424 64644 44428  5558]
 [23758 54672 39889 31523 35489]
 [48092 46383 10064 64947  6397]
 [32320 13466 31539 53025 47558]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [7]:
a.random_rotation(degrees=20)

PhiTensor(child=FixedPrecisionTensor(child=[[32320 13466 31539 53025 47558]
 [48092 46383 10064 64947  6396]
 [23758 54672 39889 31523 35489]
 [64122 55424 64644 44428  5558]
 [58499 23557 14045 58880 64741]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [8]:
a.normalize(mean=0.5, std=0.5)

PhiTensor(child=FixedPrecisionTensor(child=[[  -896 -38604  -2458  40514  29580]
 [ 30648  27230 -45408  64358 -52742]
 [-18020  43808  14242  -2490   5442]
 [ 62708  45312  63752  23320 -54420]
 [ 51462 -18422 -37446  52224  63946]]), min_vals=<lazyrepeatarray data: -1.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 1.0 -> shape: ()>)

In [9]:
a.child.decode()

array([[0.49316406, 0.20547485, 0.48124695, 0.80909729, 0.72567749],
       [0.73382568, 0.70774841, 0.15356445, 0.99101257, 0.09761047],
       [0.36251831, 0.83422852, 0.60865784, 0.48100281, 0.54151917],
       [0.97842407, 0.84570312, 0.98638916, 0.67791748, 0.08480835],
       [0.8926239 , 0.35945129, 0.21430969, 0.8984375 , 0.98786926]])

<hr>
<hr>

## Model Training


To Do:
- Implement the layers needed for a ConvNet for DP Tensors:
    - Conv layer
    - BatchNorm2D
    - LeakyReLU
    - AvgPool2d
    - Linear
- Do 1 forward pass with these layers
- Do 1 backprop with these layers

In [2]:
from syft.core.tensor.nn.conv_layers import Conv2d
import torch

In [3]:
import cv2

img = cv2.imread("/home/e/Downloads/Dataset/10253/0/10253_idx5_x1001_y1001_class0.png")
print(img.shape)
img = cv2.resize(img, (50, 50))
print(img.shape, type(img))

(50, 50, 3)
(50, 50, 3) <class 'numpy.ndarray'>


In [4]:
dp_tensor = PhiTensor(child=img, data_subjects=[0]  , min_vals=0, max_vals=255)

In [5]:
r = Conv2d(dp_tensor, in_channels=3, out_channels=32, kernel_size=3, padding=2)

In [6]:
from syft.core.tensor.nn.batch_norm import BatchNorm2d

In [7]:
img.shape

(50, 50, 3)

In [8]:
torch.nn.BatchNorm2d(3)(torch.Tensor(img.reshape(1, *img.shape[::-1]))).shape

torch.Size([1, 3, 50, 50])

In [9]:
dpt2 = dp_tensor.copy()
dpt2.child = dp_tensor.child.reshape(1, *dp_tensor.shape[::-1])
BatchNorm2d(dpt2, 3).child.decode().shape

(1, 3, 50, 50)

In [10]:
torch.nn.BatchNorm2d(3)(torch.Tensor(img.reshape(1, *img.shape[::-1])))

tensor([[[[ 0.3714, -0.8791,  0.9668,  ...,  0.7882, -0.1645, -1.5639],
          [ 0.9073, -0.4027, -1.8914,  ...,  1.2943,  1.5623,  1.6814],
          [ 1.5028,  1.6516,  1.6516,  ...,  1.2348,  0.5798,  1.4730],
          ...,
          [-1.5043,  0.8775, -0.1943,  ..., -0.1348, -1.5639,  0.4012],
          [-0.0157, -1.4150,  0.8775,  ...,  1.0562,  0.4309, -0.7004],
          [ 1.0562,  0.5500, -0.5813,  ..., -0.9088,  1.0859,  0.1630]],

         [[-0.9313,  1.0406,  0.5924,  ..., -0.2143, -1.5886,  0.3534],
          [-0.2740, -1.8276,  0.4729,  ...,  1.1003,  0.5625, -0.5429],
          [ 1.1003,  0.7119, -0.3039,  ..., -0.5130,  1.1302,  0.3833],
          ...,
          [ 0.8912, -0.2143, -1.7978,  ..., -2.0368, -0.9014,  0.2040],
          [-1.2002,  0.3235, -0.1246,  ..., -1.2002, -2.8733, -1.4691],
          [-0.3338, -1.8276, -0.0948,  ...,  0.6820,  0.0546, -1.4990]],

         [[ 0.8778,  0.6089, -0.4665,  ..., -1.8332, -0.6906, -0.3993],
          [-1.8108,  0.3625, -

In [12]:
BatchNorm2d(dpt2, 3).child.decode()

array([[[[ 0.37138367, -0.8790741 ,  0.96684265, ...,  0.78820801,
          -0.16452026, -1.56384277],
         [ 0.9072876 , -0.4026947 , -1.89134216, ...,  1.29434204,
           1.56230164,  1.68139648],
         [ 1.50274658,  1.65161133,  1.65161133, ...,  1.23480225,
           0.57978821,  1.47297668],
         ...,
         [-1.50430298,  0.8775177 , -0.19429016, ..., -0.13475037,
          -1.56384277,  0.40115356],
         [-0.01565552, -1.41497803,  0.8775177 , ...,  1.05615234,
           0.43092346, -0.70042419],
         [ 1.05615234,  0.55001831, -0.5813446 , ..., -0.90884399,
           1.0859375 ,  0.16296387]],

        [[-0.93130493,  1.04057312,  0.59240723, ..., -0.21426392,
          -1.58860779,  0.35339355],
         [-0.27401733, -1.82762146,  0.47290039, ...,  1.10032654,
           0.56253052, -0.54290771],
         [ 1.10032654,  0.71191406, -0.30389404, ..., -0.51303101,
           1.13020325,  0.38327026],
         ...,
         [ 0.89117432, -0.21426392

In [3]:
from syft import nn